## Get the dataset

<div class="alert alert-block alert-info">
https://www.kaggle.com/datasets/thedevastator/drug-performance-evaluation?select=Drug.csv and https://www.kaggle.com/datasets/thedevastator/drug-performance-evaluation?select=Drug_clean.csv
The datasets are scraped from https://www.webmd.com/ which contains a lot of information about health. 


Drug.csv :
- Condition : The medical condition associated with the drug. (String)
- Drug : The name of the drug. (String)
- Indication : The purpose of the drug. (String)
- Type : The type of drug (generic or brand). (String)
- RX : medical prescription
- OTC : a medicine that can be bought without a prescription
- Reviews : The number of reviews associated with the drug. (Integer) (object in csv)
- Effective : The effectiveness of the drug based on customer reviews. (Integer)
- EaseOfUse : The ease of use of the drug based on customer reviews. (Integer)
- Satisfaction : The satisfaction level of the drug based on customer reviews. (Integer)
- Information : Other relevant information about the drug. (String)


Drug_clean.csv :
- Condition : The medical condition associated with the drug. (String)
- Drug : The name of the drug. (String)
- Indication : The purpose of the drug. (String)
- Type : The type of drug (generic or brand). (String)
- Reviews : The number of reviews associated with the drug. (Integer)
- Effective : The effectiveness of the drug based on customer reviews. (Integer)
- EaseOfDrug : The ease of use of the drug based on customer reviews. (Integer)
- Satisfaction : The satisfaction level of the drug based on customer reviews. (Integer)
- Form : The form of the drug (e.g. tablet, capsule, etc.). (String)
- Price : The average price of the drug. (Float)
</div>

# 📚Import librairies

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from collections import Counter

# 💾Load datasets

### Read csv files

In [2]:
drug = pd.read_csv("data/Drug.csv", sep=";", header=1) #first dataset
drug_clean = pd.read_csv("data/Drug_clean.csv", sep=";", header=1) #second dataset

<div class="alert alert-block alert-info">
Sometimes we get a memory error, if so we need to execute these lines. 
</div>

In [3]:
drug = pd.read_csv("data/Drug.csv", sep=";", header=1, low_memory=False) #first dataset
drug_clean = pd.read_csv("data/Drug_clean.csv", sep=";", header=1, low_memory=False) #second dataset

<div class="alert alert-block alert-info">
When we open the datasets in Excel we see that the second row is the name avec the columns so we put this row as header. The first row is the number of columns, we can forget this line. The delimiter used as default by pandas is ",", but here we need to modify it with ";".
</div>

In [4]:
print(drug.head())
print("------------------------------------------")
print(drug_clean.head())

                   Condition          Drug Indication Type      Reviews  \
0  Acute Bacterial Sinusitis  Levofloxacin   On Label   RX  994 Reviews   
1  Acute Bacterial Sinusitis  Levofloxacin   On Label   RX  994 Reviews   
2  Acute Bacterial Sinusitis  Moxifloxacin   On Label   RX  755 Reviews   
3  Acute Bacterial Sinusitis  Azithromycin   On Label   RX  584 Reviews   
4  Acute Bacterial Sinusitis  Azithromycin   On Label   RX  584 Reviews   

   Effective  EaseOfUse  Satisfaction  \
0       2.52       3.01          1.84   
1       2.52       3.01          1.84   
2       2.78       3.00          2.08   
3       3.21       4.01          2.57   
4       3.21       4.01          2.57   

                                         Information  
0  \r\n\t\t\t\t\tLevofloxacin is used to treat a ...  
1  \r\n\t\t\t\t\tLevofloxacin is used to treat a ...  
2  \r\n\t\t\t\t\t This is a generic drug. The ave...  
3  \r\n\t\t\t\t\tAzithromycin is an antibiotic (m...  
4  \r\n\t\t\t\t\tAzithromyc

### Get info about the datasets

In [5]:
# datasets shape
print(f"drug dataset shape is {drug.shape}")
print(f"drug_clean dataset shape is {drug_clean.shape}")

drug dataset shape is (2219, 9)
drug_clean dataset shape is (685, 10)


In [6]:
# datasets columns
print(f"columns of the drug dataset are {drug.columns}")
print(f"columns of the drug_clean dataset are {drug_clean.columns}")

columns of the drug dataset are Index(['Condition', 'Drug', 'Indication', 'Type', 'Reviews', 'Effective',
       'EaseOfUse', 'Satisfaction', 'Information'],
      dtype='object')
columns of the drug_clean dataset are Index(['Condition', 'Drug', 'EaseOfUse', 'Effective', 'Form', 'Indication',
       'Price', 'Reviews', 'Satisfaction', 'Type'],
      dtype='object')


<div class="alert alert-block alert-warning">  
"Information" is in the first dataset only and "Price" and "Form" are only in the second dataset. After reading some of the rows in 'Information', we know that some information about the price and the form are written there.
</div>

In [7]:
# get infos (name, number of non-null and type of colums)
print(drug.info())
print(drug_clean.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2219 entries, 0 to 2218
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Condition     2219 non-null   object 
 1   Drug          2219 non-null   object 
 2   Indication    2219 non-null   object 
 3   Type          2219 non-null   object 
 4   Reviews       2219 non-null   object 
 5   Effective     2219 non-null   float64
 6   EaseOfUse     2219 non-null   float64
 7   Satisfaction  2219 non-null   float64
 8   Information   2219 non-null   object 
dtypes: float64(3), object(6)
memory usage: 156.1+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 685 entries, 0 to 684
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Condition     685 non-null    object 
 1   Drug          685 non-null    object 
 2   EaseOfUse     685 non-null    float64
 3   Effective     685 non-null    float64

<div class="alert alert-block alert-warning">  
We can see that the type column "Reviews" is not the same, we have float64 and object.</div>

In [8]:
print(drug.describe())
print(drug_clean.describe())

         Effective    EaseOfUse  Satisfaction
count  2219.000000  2219.000000   2219.000000
mean      3.557972     3.958824      3.218774
std       1.113128     1.037877      1.230933
min       1.000000     1.000000      1.000000
25%       3.000000     3.540000      2.400000
50%       3.680000     4.100000      3.130000
75%       4.330000     5.000000      4.000000
max       5.000000     5.000000      5.000000
        EaseOfUse   Effective         Price      Reviews  Satisfaction
count  685.000000  685.000000    685.000000   685.000000    685.000000
mean     3.920038    3.523530    174.211183    82.644410      3.195699
std      0.894212    0.954126    667.743466   273.281398      1.030442
min      1.000000    1.000000      4.000000     1.000000      1.000000
25%      3.556667    3.000000     15.490000     3.000000      2.575000
50%      4.050000    3.600000     49.990000    10.350877      3.200000
75%      4.500000    4.110000    145.990000    57.000000      3.901250
max      5.000000 

<div class="alert alert-block alert-info">
If we look in detail into the common columns we can see that the statistics  are similar which is a good thing. It means that the info are coherent.
</div>

# ⚒️Pre-processing

### 'Reviews' column

<div class="alert alert-block alert-info">
Let's begin with the column 'Reviews'.
We need to clean these columns so they can be the same type.

We think that the best way to do it is to delete the word 'Reviews' and transform the floats into integers as it is written in the description of the datasets. We will have to change the type of the column.
</div>

In [9]:
print("before pre-processing")
print(drug.Reviews[0:5])
print(drug_clean.Reviews[6:11])

print("\nNow let's split the object type (default separator is any whitespace) and keep the first part which is the number. To make sure, we are going to set the type as 'int' for both of the datasets.\n")
drug['Reviews'] = drug['Reviews'].apply(lambda x : int(x.split()[0]))    
drug = drug.astype({"Reviews": int})
drug_clean = drug_clean.astype({"Reviews": int})

print("after pre-processing")
print(drug.Reviews[0:5])
print(drug_clean.Reviews[6:11])

before pre-processing
0    994 Reviews
1    994 Reviews
2    755 Reviews
3    584 Reviews
4    584 Reviews
Name: Reviews, dtype: object
6     490.000000
7     141.600000
8      41.000000
9       3.666667
10      5.666667
Name: Reviews, dtype: float64

Now let's split the object type (default separator is any whitespace) and keep the first part which is the number. To make sure, we are going to set the type as 'int' for both of the datasets.

after pre-processing
0    994
1    994
2    755
3    584
4    584
Name: Reviews, dtype: int32
6     490
7     141
8      41
9       3
10      5
Name: Reviews, dtype: int32


<div class="alert alert-block alert-success">  
The column 'Reviews' is now well cleaned.    
</div>

### 'Information' column

<div class="alert alert-block alert-info">
Now let's focus on the 'Information' column. We can get the price, the discount and the form of the drug.
</div>

In [10]:
print(f"There are {drug['Information'].nunique()} different values in 'Information' column")

There are 421 different values in 'Information' column


In [11]:
print(f"These values are : {drug['Information'].unique()}")

These values are : ['\r\n\t\t\t\t\tLevofloxacin is used to treat a variety of bacterial infections. This is a generic drug. The average cash price for 10 Tablet(s), 500mg each of the generic (levofloxacin) is $172.99. You can buy levofloxacin at the discounted price of $47.08 by using the WebMDRx coupon, a savings of 73%. Even if this drug is covered by Medicare or your insurance, we recommend you compare prices. The WebMDRx coupon or cash price may be less than your co-pay.\r\n\t\t\t\t'
 '\r\n\t\t\t\t\t This is a generic drug. The average cash price for 1 Bottle, 3ml of 0.5% each of the generic (moxifloxacin hcl) is $74.99. You can buy moxifloxacin hcl at the discounted price of $34.89 by using the WebMDRx coupon, a savings of 53%. Even if this drug is covered by Medicare or your insurance, we recommend you compare prices. The WebMDRx coupon or cash price may be less than your co-pay.\r\n\t\t\t\t'
 '\r\n\t\t\t\t\tAzithromycin is an antibiotic (macrolide-type). This is a generic drug. 

<div class="alert alert-block alert-info">
Let's see how many prices by row we can get.
</div>

In [12]:
def number_of_prices(row):
    price = len(re.findall("\$(\d+\.\d*)", row['Information']))
    return price

drug['Nb_Prices'] = drug.apply(number_of_prices, axis=1, result_type='expand')
print(f"The number of prices we can get is in {drug['Nb_Prices'].unique()}")
print(drug['Nb_Prices'].describe())

drug_clean['Nb_Prices'] = 1

The number of prices we can get is in [2 1 0]
count    2219.000000
mean        1.863452
std         0.351235
min         0.000000
25%         2.000000
50%         2.000000
75%         2.000000
max         2.000000
Name: Nb_Prices, dtype: float64


<div class="alert alert-block alert-info">  
We can see that we can get 0, 1 or 2 prices by row. We decide to create 2 columns with the first price in 'Price' (same name as the other dataset) and the second price in 'Price2'. The column(s) will be set to NaN if there is only one price or none. We decide to not set the price at 0 because it will bias the statistics. We also add a column named 'Nb_Prices' in the second dataset and set every rows at 1.
</div>

In [13]:
def get_new_info(row):
    prices = re.findall("\$(\d+\.\d*)", row['Information'])
    if len(prices) == 2:
        price1 = float(prices[0])
        price2 = float(prices[1])
        discount = round((1-(price2/price1)) * 100, 2)
    elif len(prices) == 1:
        price1 = float(prices[0])
        price2 = np.nan
        discount = np.nan
    elif len(prices) == 0:
        price1 = np.nan
        price2 = np.nan
        discount = np.nan
        
    return price1, price2, discount

drug[['Price', 'Price2', 'Discount(%)']] = drug.apply(get_new_info, axis=1, result_type='expand')

In [14]:
print(drug['Price'].describe())
print(drug_clean['Price'].describe())

count     2213.000000
mean       111.189584
std        489.581768
min          2.590000
25%         11.990000
50%         20.620000
75%         79.460000
max      10362.190000
Name: Price, dtype: float64
count      685.000000
mean       174.211183
std        667.743466
min          4.000000
25%         15.490000
50%         49.990000
75%        145.990000
max      10362.190000
Name: Price, dtype: float64


<div class="alert alert-block alert-info">
After we print the statistics of 'Price' for both of the datasets we can see that they are similar.
</div>

In [15]:
drug.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2219 entries, 0 to 2218
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Condition     2219 non-null   object 
 1   Drug          2219 non-null   object 
 2   Indication    2219 non-null   object 
 3   Type          2219 non-null   object 
 4   Reviews       2219 non-null   int32  
 5   Effective     2219 non-null   float64
 6   EaseOfUse     2219 non-null   float64
 7   Satisfaction  2219 non-null   float64
 8   Information   2219 non-null   object 
 9   Nb_Prices     2219 non-null   int64  
 10  Price         2213 non-null   float64
 11  Price2        1922 non-null   float64
 12  Discount(%)   1922 non-null   float64
dtypes: float64(6), int32(1), int64(1), object(5)
memory usage: 216.8+ KB


<div class="alert alert-block alert-info">
We can see that the 3 new columns have been well added to the dataset.
</div>

<div class="alert alert-block alert-success">  
Now we have the price(s) and the discount.
</div>

<div class="alert alert-block alert-info">  
Let's get the form of the drug. First let's get an idea of what forms exist in the dataset where they are provided.
</div>

In [30]:
print(f"The form of drug can be {drug_clean['Form'].unique()}")

The form of drug can be ['capsule' 'liquid (drink)' 'tablet' 'cream' 'liquid (inject)' 'other']


<div class="alert alert-block alert-info">  
We see that there are 6 types of form. We did not exclude that it can be others in the other dataset. Now we are going to search these types in the 'Information' column. To avoid syntax errors, we decide to apply .lower() to the types of form and the 'Information' column.
</div>

In [340]:
drug_clean['Form'] = drug_clean['Form'].str.lower()
forms = list(drug_clean['Form'].unique())
print(f"Let's search for these forms : {forms}")

Let's search for these forms : ['capsule', 'liquid (drink)', 'tablet', 'cream', 'liquid (inject)', 'other']


In [341]:
drug['Information'] = drug['Information'].str.lower()

def get_forms(row):
    if f in row['Information']:
        print(row['Information'])
        print(f)
        return f
    else: return np.nan

for f in forms:
    drug['Form'] = drug.apply(get_forms, axis=1, result_type='expand')


					amoxicillin is used to treat a wide variety of bacterial infections. this is a generic drug. the average cash price for 21 capsule(s), 500mg  each, is $12.59. you can buy amoxicillin at the discounted price of $4.78 by using the webmdrx coupon, a savings of 62%. even if this drug is covered by medicare or your insurance, we recommend you compare prices. the webmdrx coupon or cash price may be less than your co-pay.
				
capsule

					amoxicillin is used to treat a wide variety of bacterial infections. this is a generic drug. the average cash price for 21 capsule(s), 500mg  each, is $12.59. you can buy amoxicillin at the discounted price of $4.78 by using the webmdrx coupon, a savings of 62%. even if this drug is covered by medicare or your insurance, we recommend you compare prices. the webmdrx coupon or cash price may be less than your co-pay.
				
capsule

					amoxicillin is used to treat a wide variety of bacterial infections. this is a generic drug. the average cash price fo

<div class="alert alert-block alert-danger">  
We can see that the value 'other' is ambiguous and we should not search for it. Here is one critical example : "Tacrolimus is used with other medications to prevent rejection of a kidney....", we don't want to count it. We delete 'other' of the array that we iterate through and create a list for saving the index of the row where no form is found.
</div>

In [342]:
#let's delete 'other'
forms.pop(-1)

found_forms = []

def get_forms2(row):
    for f in forms:
        if f in row['Information']:
            found_forms.append(f)
            return f

    return np.nan
        
drug['Form'] = drug.apply(get_forms2, axis=1, result_type='expand')

found_forms = list(set(found_forms))
print(f"There are {len(found_forms)} forms of drugs found")

There are 3 forms of drugs found


<div class="alert alert-block alert-info">
Only 3 forms of drugs have been found in the column. Let's see which ones.
</div>

In [344]:
drug['Form'].unique()

array(['tablet', nan, 'capsule', 'cream'], dtype=object)

<div class="alert alert-block alert-info">
We have seen that we only get tablet, capsule and cream forms. We'll have to deal with the other forms.
To get them, we are going to use regex to find every word written after an integer and a whitespace. In the first time, we are just going to create a Counter object. The latter will provides the number of occurrences of each of the word found. If there are few words, then we will be able to treat them manually and keep the ones we want.
</div>

In [345]:
array_forms = []

def get_new_forms(row):
    result = re.findall(r"\d(\s\w*)", row['Information'])
    [array_forms.append(item_result.strip()) for item_result in result if (item_result not in array_forms) and (len(result) != 0)]
        
drug[drug['Form'].isnull()].apply(get_new_forms, axis=1, result_type='expand')

c = Counter(array_forms)
print(c.most_common())

[('by', 891), ('tube', 235), ('bottle', 209), ('box', 40), ('vial', 38), ('solution', 27), ('suppository', 21), ('can', 13), ('suspension', 10), ('syringe', 10), ('patch', 10), ('at', 7), ('pen', 7), ('', 5), ('each', 5), ('package', 5), ('packet', 4), ('jar', 3), ('kit', 2), ('ingredients', 2), ('days', 2), ('inhaler', 2), ('aerosol', 2), ('implant', 1)]


In [347]:
print(f"There are {drug[drug['Form'].isnull()].shape[0]} rows without any forms (np.nan)")

There are 897 rows without any forms (np.nan)


<div class="alert alert-block alert-info">
We can see that some of the words found can easily be forgotten, for example "by", "", "days", "at", "each" and "ingredients". Let's delete them and count the occurrences of all the other words. 
</div>

In [348]:
for to_delete in ["by", "", "days", "at", "each", "ingredients"]:
    array_forms = list(filter((to_delete).__ne__, array_forms))
c = Counter(array_forms)
print(c.most_common())

[('tube', 235), ('bottle', 209), ('box', 40), ('vial', 38), ('solution', 27), ('suppository', 21), ('can', 13), ('suspension', 10), ('syringe', 10), ('patch', 10), ('pen', 7), ('package', 5), ('packet', 4), ('jar', 3), ('kit', 2), ('inhaler', 2), ('aerosol', 2), ('implant', 1)]


<div class="alert alert-block alert-info">
At the end, we decide to keep all of the other words and do an .apply() to the dataset to write the form.
</div>

In [349]:
found_forms = []
new_forms = [i[0] for i in c.most_common()]

def get_new_forms(row):
    try: 
        if np.isnan(row['Form']) :
            for f in new_forms:
                if f in row['Information']:
                    found_forms.append(f)
                    return f
            return np.nan

    except:
        return row['Form']
        
drug['Form'] = drug.apply(get_new_forms, axis=1, result_type='expand')

In [350]:
drug['Form'].unique()

array(['tablet', 'bottle', 'can', 'capsule', 'vial', 'solution', 'tube',
       'cream', 'box', nan], dtype=object)

In [352]:
print(f"There are {drug[drug['Form'].isnull()].shape[0]} rows without any forms (np.nan)")

There are 6 rows without any forms (np.nan)
